Ancien model : Tentative d'exexcution d'un model VGG16 et Embeddings NLP en simultané

Code à finaliser, ne fonctionne plus

In [2]:
import numpy as np
import pandas as pd

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, GlobalAveragePooling2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam,Adagrad

from keras.applications.vgg16 import VGG16
from keras.applications import VGG19
from keras.applications import EfficientNetB0




from tensorflow.keras.applications.vgg16 import preprocess_input
import tensorflow as tf

from RktnChallenge.RktnModel import ModelTrainer


# PARAMS  de base
PATH = "images/crop224/"
img_size = 224
depth = 3

from RktnChallenge.RktnModel import ModelTrainer

trainer = ModelTrainer("data.csv",PATH)

list_tags = trainer.data.prdtypecode.unique().tolist()
num_categories = len(list_tags)
nb_Sample = 100

In [5]:

from RktnChallenge.RktnModel import ModelTrainer

trainer = ModelTrainer("data_tr_lemm.csv",PATH)


list_tags = trainer.data.prdtypecode.unique().tolist()
num_categories = len(list_tags)



y = trainer.encoder.fit_transform(trainer.data["prdtypecode"])



In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

class MultiInputDataGenerator:
    def __init__(self, image_data_gen: ImageDataGenerator):
        self.image_data_gen = image_data_gen

    def flow_from_dataframe(self, dataframe, x_col_img, x_col_emb, y_col, target_size, batch_size,classes):
        # Image generator
        img_gen = self.image_data_gen.flow_from_dataframe(
            dataframe,
            x_col=x_col_img,
            y_col=y_col,
            target_size=target_size,
            batch_size=batch_size,
            class_mode="categorical",
            seed=42,
            classes = classes,
        )
        
        
        while True:
            img_batch, label_batch = next(img_gen)
            idx = (img_gen.batch_index - 1) * img_gen.batch_size
            emb_batch = dataframe[x_col_emb].iloc[idx: idx + batch_size].to_numpy()
            emb_batch = np.array(emb_batch.tolist())
            yield [img_batch, emb_batch], label_batch




preprocessing_function = tf.keras.applications.vgg16.preprocess_input
image_data_gen = ImageDataGenerator(preprocessing_function = preprocessing_function)
multi_gen = MultiInputDataGenerator(image_data_gen)


from sklearn.model_selection import train_test_split

# Supposons que df est votre dataframe
train_df, temp_df = train_test_split(df_copy, test_size=0.3, random_state=42)
valid_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

batch_size = 20


train_generator = multi_gen.flow_from_dataframe(
    dataframe=train_df,
    x_col_img="imgname",
    x_col_emb="tokens",
    y_col="prdtypecode",
    target_size=(img_size, img_size),
    batch_size=batch_size,
    classes=classes
)
valid_generator = multi_gen.flow_from_dataframe(
    dataframe=valid_df,
    x_col_img="imgname",
    x_col_emb="tokens",
    y_col="prdtypecode",
    target_size=(img_size, img_size),
    batch_size=batch_size,
    classes=classes
)
test_generator = multi_gen.flow_from_dataframe(
    dataframe=valid_df,
    x_col_img="imgname",
    x_col_emb="tokens",
    y_col="prdtypecode",
    target_size=(img_size, img_size),
    batch_size=batch_size,
    classes=classes
)


In [12]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Embedding, LSTM, GlobalAveragePooling2D
from keras.applications.vgg16 import VGG16
from keras.applications import VGG19
from keras.applications import EfficientNetB0
from tensorflow.keras.models import Model

# La partie Image
input_image = Input(shape=(img_size, img_size, depth), name="image_input")
net = VGG16(weights='imagenet', include_top=False) 
x_image = net(input_image)

for layer in net.layers[:-2]:
        layer.trainable = False
        
x_image = tf.keras.layers.GlobalAveragePooling2D()(x_image)
x_image = tf.keras.layers.BatchNormalization()(x_image)
x_image = Dense(512, activation='relu')(x_image)
x_image = Dropout(0.3)(x_image)


# La partie Texte
max_words = trainer.max_seq_length
vocab_size = trainer.vocab_size  # Total vocabulary size
embedding_dim = 600  # Embedding dimensions

input_text = Input(shape=(max_words,), name="text_input")
x_text = Embedding(vocab_size, embedding_dim)(input_text)
x_text = Flatten()(x_text)

# Le merge des deux towers
merged = tf.keras.layers.concatenate([x_image, x_text])

output = Dense(27, activation='softmax')(merged)


model = Model(inputs=[input_image, input_text], outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_input (InputLayer)    [(None, 100, 100, 3)]        0         []                            
                                                                                                  
 vgg16 (Functional)          (None, None, None, 512)      1471468   ['image_input[0][0]']         
                                                          8                                       
                                                                                                  
 global_average_pooling2d_2  (None, 512)                  0         ['vgg16[0][0]']               
  (GlobalAveragePooling2D)                                                                        
                                                                                            

Avec pondération

In [9]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Embedding, LSTM, GlobalAveragePooling2D, Lambda
from keras.applications.vgg16 import VGG16
from keras.applications import VGG19
from keras.applications import EfficientNetB0
from tensorflow.keras.models import Model

# La partie Image
input_image = Input(shape=(img_size, img_size, depth), name="image_input")
net = VGG16(weights='imagenet', include_top=False) 
x_image = net(input_image)

for layer in net.layers[:]:
        layer.trainable = False
         
x_image = tf.keras.layers.GlobalAveragePooling2D()(x_image)
x_image = tf.keras.layers.BatchNormalization()(x_image)
x_image = Dense(512, activation='relu')(x_image)
x_image = Dropout(0.3)(x_image)


# La partie Texte
max_words = trainer.max_seq_length
vocab_size = trainer.vocab_size  # Total vocabulary size
embedding_dim = 150  # Embedding dimensions

input_text = Input(shape=(max_words,), name="text_input")
x_text = Embedding(vocab_size, embedding_dim)(input_text)
x_text = Flatten()(x_text)
#x_text = LSTM(128)(x_text)
x_text = Dense(512, activation='relu')(x_text)
x_text = Dropout(0.3)(x_text)

# Le merge des deux towers
# Poids pour la moyenne pondérée
w_img = tf.Variable(initial_value=tf.ones([512])*0.5, trainable=True, dtype=tf.float32) # initialisé à 0.5, mais entraînable
w_emb = tf.Variable(initial_value=tf.ones([512])*0.5, trainable=True, dtype=tf.float32) # initialisé à 0.5, mais entraînable

# Fusion moyenne pondérée
moyenne_features = Lambda(lambda features: w_img * features[0] + w_emb * features[1])([x_image, x_text])
x_merge = Dense(128, activation='relu')(moyenne_features)
output = Dense(27, activation='softmax')(x_merge)


model = Model(inputs=[input_image, input_text], outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


The following Variables were used a Lambda layer's call (lambda), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(512,) dtype=float32>
  <tf.Variable 'Variable:0' shape=(512,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_input (InputLayer)    [(None, 100, 100, 3)]        0         []                            
                                                                                                  
 vgg16 (Functional)          (None, None, None, 512)      1471468   ['image_input[0][0]']         
                                                          8          

test

In [13]:
model.fit(
    X_train,y_train,
    validation_split=0.2,
    epochs=1
)

ValueError: in user code:

    File "/home/pec/miniconda3/envs/tfGPU/lib/python3.9/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/home/pec/miniconda3/envs/tfGPU/lib/python3.9/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/pec/miniconda3/envs/tfGPU/lib/python3.9/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/home/pec/miniconda3/envs/tfGPU/lib/python3.9/site-packages/keras/src/engine/training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "/home/pec/miniconda3/envs/tfGPU/lib/python3.9/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/pec/miniconda3/envs/tfGPU/lib/python3.9/site-package

In [41]:
model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=1
)



   2961/Unknown - 77s 26ms/step - loss: 0.8589 - accuracy: 0.7255

2023-08-30 23:30:26.001819: W tensorflow/core/framework/op_kernel.cc:1816] INVALID_ARGUMENT: TypeError: `generator` yielded an element of shape (0,) where an element of shape (None, None) was expected.
Traceback (most recent call last):

  File "/home/pec/miniconda3/envs/tfGPU/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 268, in __call__
    ret = func(*args)

  File "/home/pec/miniconda3/envs/tfGPU/lib/python3.9/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/home/pec/miniconda3/envs/tfGPU/lib/python3.9/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 235, in generator_py_func
    raise TypeError(

TypeError: `generator` yielded an element of shape (0,) where an element of shape (None, None) was expected.


2023-08-30 23:30:26.048887: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 13532273789714353523
2023-08-30 23:

InvalidArgumentError: Graph execution error:

2 root error(s) found.
  (0) INVALID_ARGUMENT:  TypeError: `generator` yielded an element of shape (0,) where an element of shape (None, None) was expected.
Traceback (most recent call last):

  File "/home/pec/miniconda3/envs/tfGPU/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 268, in __call__
    ret = func(*args)

  File "/home/pec/miniconda3/envs/tfGPU/lib/python3.9/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/home/pec/miniconda3/envs/tfGPU/lib/python3.9/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 235, in generator_py_func
    raise TypeError(

TypeError: `generator` yielded an element of shape (0,) where an element of shape (None, None) was expected.


	 [[{{node PyFunc}}]]
	 [[IteratorGetNext]]
	 [[IteratorGetNext/_4]]
  (1) INVALID_ARGUMENT:  TypeError: `generator` yielded an element of shape (0,) where an elem